 <div style="background-color:#004d99; color:white; padding:30px; BORDER-radius:10px; text-align:center;">
  <h1 style="margin:0; font-size:1.8em;">ADVANCED DATA ANALYTICS DASHBOARD</h1>
</div>

In [3]:
import subprocess
import time
import os
import signal
import sys

# Streamlit app code
app_code = '''

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set page configuration
st.set_page_config(
    page_title="Advanced Data Analytics Dashboard",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for styling
st.markdown("""
<style>
    /* Main styling */
    .main .block-container {
        background-color: pink;
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    
    /* Header styling */
    .header {
        display: flex;
        justify-content: space-between;
        align-items: center;
        padding: 1rem 0;
        margin-bottom: 1rem;
    }
    
    /* Card styling */
    .card {
        background-color: white;
        border-radius: 10px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        padding: 1.5rem;
        margin-bottom: 1.5rem;
        height: 100%;
    }
    
    .metric-card {
        text-align: center;
        padding: 1rem;
    }
    
    .metric-value {
        font-size: 1.8rem;
        font-weight: 700;
        color: #2c3e50;
        margin: 0.5rem 0;
    }
    
    .metric-label {
        font-size: 0.9rem;
        color: #7f8c8d;
        margin-bottom: 0.5rem;
    }
    
    .metric-change {
        font-size: 0.8rem;
        display: flex;
        align-items: center;
        justify-content: center;
    }
    
    .positive-change {
        color: #2ecc71;
    }
    
    .negative-change {
        color: #e74c3c;
    }
    
    /* Section headers */
    .section-header {
        font-size: 1.2rem;
        font-weight: 600;
        color: #2c3e50;
        margin-bottom: 1rem;
        padding-bottom: 0.5rem;
        border-bottom: 1px solid #eaeaea;
    }
    
    /* Filter section */
    .filter-section {
        background-color: #f8f9fa;
        padding: 1rem;
        border-radius: 10px;
        margin-bottom: 1.5rem;
    }
    
    /* Responsive adjustments */
    @media (max-width: 1150px) {
        .metric-value {
            font-size: 1.5rem;
        }
    }
</style>
""", unsafe_allow_html=True)

# Load data from CSV file
@st.cache_data
def load_data(uploaded_file):
    try:
        # Try to detect the file format and read accordingly
        if uploaded_file.name.endswith('.csv'):
            df = pd.read_csv(uploaded_file)
        elif uploaded_file.name.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(uploaded_file)
        else:
            st.error("Unsupported file format. Please upload a CSV or Excel file.")
            return None
        
        # Convert date columns if they exist
        date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
        
        st.success(f"Data loaded successfully! Shape: {df.shape[0]} rows, {df.shape[1]} columns")
        return df
        
    except Exception as e:
        st.error(f"Error loading data: {str(e)}")
        return None

# File upload section
st.sidebar.header('Data Upload')
uploaded_file = st.sidebar.file_uploader(
    "Upload your dataset (CSV or Excel)", 
    type=['csv', 'xlsx', 'xls']
)

if uploaded_file is not None:
    df = load_data(uploaded_file)
    
    if df is not None:
        # Display column information
        st.sidebar.subheader("Dataset Information")
        st.sidebar.write(f"**Rows:** {df.shape[0]}")
        st.sidebar.write(f"**Columns:** {df.shape[1]}")
        
        # Show column names and types
        if st.sidebar.checkbox("Show column details"):
            col_info = pd.DataFrame({
                'Column': df.columns,
                'Type': df.dtypes.values,
                'Non-Null Count': df.notnull().sum().values
            })
            st.sidebar.dataframe(col_info)
        
        # Sidebar filters
        st.sidebar.header('Filters')
        
        # Date range filter (if date column exists)
        date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
        if date_columns:
            date_col = st.sidebar.selectbox("Select date column", date_columns)
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
            
            min_date = df[date_col].min()
            max_date = df[date_col].max()
            
            date_range = st.sidebar.date_input(
                "Select Date Range",
                value=(min_date, max_date),
                min_value=min_date,
                max_value=max_date
            )
            
            if len(date_range) == 2:
                start_date, end_date = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])
                filtered_df = df[(df[date_col] >= start_date) & (df[date_col] <= end_date)]
            else:
                filtered_df = df
        else:
            filtered_df = df
            st.sidebar.info("No date columns found in dataset")
        
        # Categorical filters
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in categorical_cols:
            if len(df[col].unique()) <= 20:  # Only show filters for columns with reasonable unique values
                options = st.sidebar.multiselect(
                    f'Filter by {col}',
                    options=df[col].unique(),
                    default=df[col].unique()
                )
                
                if options:
                    filtered_df = filtered_df[filtered_df[col].isin(options)]
        
        # Numerical filters
        numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        for col in numerical_cols:
            if len(df[col].unique()) > 10:  # Only show sliders for columns with many unique values
                min_val = float(df[col].min())
                max_val = float(df[col].max())
                
                selected_range = st.sidebar.slider(
                    f'Range for {col}',
                    min_val, max_val, (min_val, max_val)
                )
                
                filtered_df = filtered_df[(filtered_df[col] >= selected_range[0]) & 
                                         (filtered_df[col] <= selected_range[1])]
        
        # Main dashboard layout
        st.title('📊 Advanced Data Analytics Dashboard')
        st.markdown(f"""
        This interactive dashboard provides insights into your dataset.
        **Loaded dataset:** {uploaded_file.name}
        """)
        
        # Key metrics row - dynamically created based on available numerical columns
        st.subheader('Key Metrics')
        
        # Select up to 4 numerical columns to display as metrics
        metric_cols = st.multiselect(
            "Select numerical columns to display as metrics",
            options=numerical_cols,
            default=numerical_cols[:min(4, len(numerical_cols))],
            key="metric_columns"
        )
        
        cols = st.columns(min(4, len(metric_cols)))
        
        for i, col_name in enumerate(metric_cols):
            if i < 4:  # Limit to 4 metrics
                with cols[i]:
                    total_value = filtered_df[col_name].sum()
                    avg_value = filtered_df[col_name].mean()
                    
                    st.markdown(f"""
                    <div class="card metric-card">
                        <div class="metric-label">{col_name}</div>
                        <div class="metric-value">{total_value:,.2f}</div>
                        <div class="metric-label">Average: {avg_value:,.2f}</div>
                    </div>
                    """, unsafe_allow_html=True)
        
        # Charts and visualizations
        st.subheader('Data Visualizations')
        
        # Column selection for charts
        col1, col2 = st.columns(2)
        
        with col1:
            # Select x and y axes for scatter plot
            x_axis = st.selectbox("X-axis for scatter plot", options=df.columns, index=0)
            y_axis = st.selectbox("Y-axis for scatter plot", options=numerical_cols, index=min(1, len(numerical_cols)-1))
            color_axis = st.selectbox("Color by (optional)", options=['None'] + categorical_cols)
            
            if color_axis != 'None':
                fig = px.scatter(filtered_df, x=x_axis, y=y_axis, color=color_axis, 
                                title=f'{y_axis} vs {x_axis} by {color_axis}')
            else:
                fig = px.scatter(filtered_df, x=x_axis, y=y_axis, 
                                title=f'{y_axis} vs {x_axis}')
            st.plotly_chart(fig, use_container_width=True)
        
        with col2:
            # Select column for distribution plot
            dist_col = st.selectbox("Select column for distribution", options=numerical_cols)
            
            fig = px.histogram(filtered_df, x=dist_col, title=f'Distribution of {dist_col}')
            st.plotly_chart(fig, use_container_width=True)
        
        # Additional visualizations
        st.subheader('Additional Insights')
        
        col1, col2 = st.columns(2)
        
        with col1:
            # Bar chart by category
            if categorical_cols:
                bar_category = st.selectbox("Select category for bar chart", options=categorical_cols)
                bar_value = st.selectbox("Select value for bar chart", options=numerical_cols)
                
                bar_data = filtered_df.groupby(bar_category)[bar_value].sum().reset_index()
                fig = px.bar(bar_data, x=bar_category, y=bar_value, 
                            title=f'{bar_value} by {bar_category}')
                st.plotly_chart(fig, use_container_width=True)
        
        with col2:
            # Pie chart
            if categorical_cols:
                pie_category = st.selectbox("Select category for pie chart", options=categorical_cols)
                
                pie_data = filtered_df[pie_category].value_counts().reset_index()
                pie_data.columns = [pie_category, 'count']
                
                fig = px.pie(pie_data, values='count', names=pie_category, 
                            title=f'Distribution of {pie_category}')
                st.plotly_chart(fig, use_container_width=True)
        
        # Correlation heatmap
        st.subheader('Correlation Analysis')
        
        # Select numerical columns for correlation
        corr_cols = st.multiselect(
            "Select numerical columns for correlation analysis",
            options=numerical_cols,
            default=numerical_cols[:min(6, len(numerical_cols))]
        )
        
        if len(corr_cols) > 1:
            corr_data = filtered_df[corr_cols].corr()
            
            fig = go.Figure(data=go.Heatmap(
                z=corr_data.values,
                x=corr_data.columns,
                y=corr_data.columns,
                colorscale='RdBu',
                zmin=-1,
                zmax=1,
                hoverongaps=False,
                text=corr_data.round(2).values,
                texttemplate="%{text}"
            ))
            fig.update_layout(title='Correlation Matrix', height=500)
            st.plotly_chart(fig, use_container_width=True)
        
        # Data table
        st.subheader('Filtered Data View')
        st.dataframe(filtered_df, use_container_width=True)
        
        # Download button
        csv = filtered_df.to_csv(index=False)
        st.download_button(
            label="Download Filtered Data as CSV",
            data=csv,
            file_name="filtered_data.csv",
            mime="text/csv"
        )
        
        # Footer
        st.markdown("---")
        st.markdown("### About This Dashboard")
        st.markdown("""
        This analytics dashboard provides insights into your uploaded dataset.
        Use the filters in the sidebar to explore specific segments of the data.
        """)
        
    else:
        st.error("Failed to load the dataset. Please check the file format and try again.")
else:
    st.info("👆 Please upload a CSV or Excel file to begin analysis")
    
    # Show sample data structure expectation
    st.subheader("Expected Data Format")
    st.markdown("""
    For best results, your dataset should include:
    - At least one numerical column (for metrics and charts)
    - One or more categorical columns (for filtering and grouping)
    - A date column (optional, for time-based analysis)
    
    The dashboard will automatically detect column types and adjust the available visualizations.
    """)

# Add some helpful tooltips
st.sidebar.info("💡 Tip: Upload a CSV or Excel file with numerical and categorical columns for best results.")
  
'''


# Save app
app_file = "Test_project_13.py"
with open(app_file, "w", encoding="utf-8") as f:
    f.write(app_code)

print("✅ Streamlit app saved as Test_project_13.py")

# Launch Streamlit
try:
    print("🚀 Starting Streamlit server...")
    process = subprocess.Popen([sys.executable, "-m", "streamlit", "run", app_file])
    
    # Wait for startup (adjust as needed)
    time.sleep(5)
    
    print("💡 App is running. Press Ctrl+C to stop.")
    # Keep the script alive until interrupted
    while True:
        time.sleep(1)

except KeyboardInterrupt:
    print("\n🛑 Shutting down Streamlit server...")
    if process.poll() is None:  # If still running
        process.terminate()
        try:
            process.wait(timeout=10)
        except subprocess.TimeoutExpired:
            process.kill()
            process.wait()
    
    #` Optional: remove the generated file
    os.remove(app_file)
    print("🧹 Cleaned up app file and server.")

✅ Streamlit app saved as Test_project_13.py
🚀 Starting Streamlit server...
💡 App is running. Press Ctrl+C to stop.

🛑 Shutting down Streamlit server...
🧹 Cleaned up app file and server.
